In [1]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
fname = 'pitches_2017.pkl'
df_17 = pd.read_pickle(fname, compression='zip')

#convert the pitch type for UN (unknown) to np.nan
df_17['pitch_type'] = df_17['pitch_type'].replace({'UN':np.nan})

In [3]:
def strikezone_wrangle(df):
    df = df.copy()
    
    #create swung column
    def swung(x):
        swung = ['foul','hit_into_play','swinging_strike','hit_into_play_no_out',
                 'hit_into_play_score','foul_tip','swinging_strike_blocked',
                 'foul_bunt','missed_bunt']
        return 1 if x in swung else 0
    df['batter_swung'] = df['description'].apply(swung)
    
    #initialize in_strikezone and chased features:
    df['in_strikezone'] = 1
    df['chased'] = 0
    
    #iterate thru each row
    for index, row in df.iterrows():
        #if ball is outside the strikezone, change the value for that row to 0
        if row.plate_z > row.sz_top or row.plate_z < row.sz_bot or row.plate_x < -0.73 or row.plate_x > 0.73:
            df.at[index, 'in_strikezone'] = 0
        #if batter_swung at ball outside the strike zone, change chased value to 1:
        if df.at[index, 'batter_swung'] == 1 and df.at[index, 'in_strikezone'] == 0:
            df.at[index, 'chased'] = 1
    return df

In [ ]:
%%time
df_17 = strikezone_wrangle(df_17)

In [5]:
def gen_pitcher_percentages(df):
    df = df.copy()
    #get all the unique pitcher names in the df
    pitcher_list = df.player_name.unique().tolist()
    #initialize empty dictionary to store each pitcher and their pitches and percentages for each pitch
    pitcher_dict = {}
    #iterate over each pitcher:
    for pitcher in pitcher_list:
        #assign the normalized value_counts to a variable
        pitch_percentages = df[df.player_name == pitcher].pitch_type.value_counts(normalize=True)
        #convert that Series object to a dict and assign it as the value to the pitcher dictionary 
        #(pitcher name as key)
        pitcher_dict[pitcher] = pitch_percentages.to_dict()
    return pitcher_dict

def fill_pitch_type_nans(df):
    df = df.copy()
    #grab the rows where pitch_type is null:
    nulls = df[df.pitch_type.isna()]
    
    #generate the dictionary of pitch type % by pitcher
    pitcher_dict = gen_pitcher_percentages(df)
    
    #iterate over each null row
    for index, row in nulls.iterrows():
        #use the % for that pitcher for each pitch type he throws to generate a random pitch type with 
        #that % as weight
        pitch = random.choices(population=list(pitcher_dict[row.player_name].keys()), 
                               weights=list(pitcher_dict[row.player_name].values()), 
                               k=1)[0]
        #fill the NaN value with the randomly generated pitch
        df.at[index, 'pitch_type'] = pitch
    return df

In [6]:
df_17.pitch_type.isna().sum()

2215

In [7]:
%%time
df_17 = fill_pitch_type_nans(df_17)

CPU times: user 1min, sys: 560 ms, total: 1min 1s
Wall time: 1min 1s


In [8]:
df_17.pitch_type.isna().sum()

0

In [9]:
df_17.pitch_type.value_counts()

FF    254021
SL    117668
FT     94633
CH     71742
CU     56710
SI     52400
FC     37607
KC     21399
FS     11782
KN      2690
FO       248
EP       188
PO       116
SC        39
Name: pitch_type, dtype: int64

## Categorize pitch type into fastball, breaking, or offspeed, (or pitchout)

#### Fastballs
FA = fastball, FF = four-seam fastball, FT = two-seam fastball, FC = fastball (cutter), 

FS / SI / SF = fastball (sinker, split-fingered)

#### Breaking
SL = slider, CB / CU = curveball, KC = knuckle-curve, SC = screwball

#### Offspeed
CH = changeup, KN = knuckleball, EP = eephus

#### Pitchout
PO / FO = pitch out



In [10]:
#create map for pitch type into categories:
pitch_type_map = {'FA':'fastball', 'FF':'fastball', 'FT':'fastball', 'FC':'fastball',
                  'FS':'fastball', 'SI':'fastball', 'SF':'fastball', 'SL':'breaking',
                  'CB':'breaking', 'CU':'breaking', 'SC':'breaking', 'KC':'breaking',
                  'CH':'offspeed', 'KN':'offspeed', 'EP':'offspeed', 'FO':'pitchout', 
                  'PO':'pitchout'}

#create pitch cateogory feature
df_17['pitch_cat'] = df_17['pitch_type']
df_17['pitch_cat'] = df_17['pitch_cat'].replace(pitch_type_map)

In [11]:
df_17['pitch_cat'].value_counts(dropna=False)

fastball    450443
breaking    195816
offspeed     74620
pitchout       364
Name: pitch_cat, dtype: int64

# Batter Scouting Report
## Batter features vs diff pitch type categories:

In [12]:
%%time
def make_batters_dict(df):
    df = df.copy()
    #make list of the unique batter ids
    batters = list(df['batter'].unique())
    #initialize empty dictionary to store the batter stats
    batters_dict = {}
    #set a break flag to False for error-checking
    brk = False
    #iterate thru each unique batter
    for batter in batters:
        if brk:
            break
        #make subset of the df for that batter and assign to variable batter_df    
        batter_df = df[df['batter'] == batter]
        
        #assign all pitch categories to list:
        all_pitch_cats = ['fastball', 'breaking', 'offspeed', 'pitchout']
        #assign the pitch categories to a list
        pitch_cats = batter_df['pitch_cat'].unique().tolist()
        #get the normalized value counts of pitches by category that batter has faced
        vc = batter_df.pitch_cat.value_counts(normalize=True)
        #initialize empty dict for each batter
        batter_dict = {}
        
        #if there are any pitch categories the batter has not faced, 
        unfaced_cats = list(set(all_pitch_cats) - set(pitch_cats))
        
        #assign NaNs to his dictionary for that category
        for cat in unfaced_cats:
            batter_dict[cat] = {}
            if cat == 'pitchout':
                batter_dict[cat]['perc_faced'] = 0
            else:
                batter_dict[cat]['perc_faced'] = np.nan
                batter_dict[cat]['chase_perc'] = np.nan
                batter_dict[cat]['bip_swung_perc'] = np.nan
                batter_dict[cat]['taken_strike_perc'] = np.nan
                batter_dict[cat]['est_woba'] = np.nan
                batter_dict[cat]['babip'] = np.nan
                batter_dict[cat]['iso_value'] = np.nan
        
        for cat in pitch_cats:
            if brk:
                break
            batter_dict[cat] = {}
        
            #assign the % of pitches faced by the batter for that category to his batter dict
            batter_dict[cat]['perc_faced'] = vc[cat] * 100
        
            #continue out of the loop for pitchout category since ball in play stats are NaN
            if cat == 'pitchout':
                continue
        
            #grab subset of batter df for the pitch category
            cat_df = batter_df[batter_df['pitch_cat'] == cat]
        
            #calculate batters chase % for pitch type category on balls outside the strikezone
            out_of_strikezone = len(cat_df[cat_df['in_strikezone'] == 0]) #num of times ball was out of zone
            chased_count = len(cat_df[cat_df['chased'] == 1]) #num of times batter chased
            try:
                chase_perc = (chased_count / out_of_strikezone) * 100
            except ZeroDivisionError:
                chase_perc = np.nan
            #assign the chase perc to the batter dict
            batter_dict[cat]['chase_perc'] = chase_perc
        
            #calc ball in play % for each swing for each pitch cat:
            ball_in_play_count = len(cat_df[cat_df['type'] == 'X']) #type X means ball hit into play
            swung_count = cat_df['batter_swung'].sum() #counts all the 1s in the swung column
            #assign the ball in play % per swing to the batter dict
            batter_dict[cat]['bip_swung_perc'] = (ball_in_play_count / swung_count) * 100
        
            #calculate taken strike %
            taken_strike_count = len(cat_df[(cat_df['in_strikezone'] == 1) & (cat_df['batter_swung'] == 0)])
            pitches_in_zone_count = cat_df['in_strikezone'].sum() #counts the 1s in the in zone col
            #assign to batter_dict
            batter_dict[cat]['taken_strike_perc'] = (taken_strike_count / pitches_in_zone_count) * 100
        
            #for each pitch type category, get the batters stats on balls hit in play
            stats = ['estimated_woba_using_speedangle', 'babip_value', 'iso_value']
            for stat in stats:
                #drop Nans from the stat column and assign to new subset, for each stat
                stat_cat_df = cat_df.dropna(subset=[stat])
                if stat == 'estimated_woba_using_speedangle':
                    #get the mean avg_est_woba
                    avg_est_woba = stat_cat_df['estimated_woba_using_speedangle'].mean()
                    #assign that value to the batters dictionary
                    batter_dict[cat]['est_woba'] = avg_est_woba
                    if avg_est_woba == np.nan:
                        print(batter)
                        brk = True
                        break
                elif stat == 'babip_value':
                    avg_babip = stat_cat_df['babip_value'].mean()
                    batter_dict[cat]['babip'] = avg_babip
                else:
                    avg_iso_value = stat_cat_df['iso_value'].mean()
                    batter_dict[cat]['iso_value'] = avg_iso_value
            
        #assign the batter dictionary to the main dictionary of all batters
        batters_dict[batter] = batter_dict
    if not brk:
        print('iteration completed successfully')
    return batters_dict

batters_dict = make_batters_dict(df_17)

/home/jm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in long_scalars
/home/jm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in long_scalars


iteration completed successfully
CPU times: user 55.6 s, sys: 560 ms, total: 56.2 s
Wall time: 56.2 s


#### Example batter:

In [19]:
batter = list(batters_dict.keys())[0]

batters_dict[batter]

{'pitchout': {'perc_faced': 0},
 'fastball': {'perc_faced': 62.89655172413793,
  'chase_perc': 22.134387351778656,
  'bip_swung_perc': 35.37735849056604,
  'taken_strike_perc': 23.15270935960591,
  'est_woba': 0.38004000000000004,
  'babip': 0.08547008547008547,
  'iso_value': 0.1794871794871795},
 'offspeed': {'perc_faced': 12.551724137931034,
  'chase_perc': 27.27272727272727,
  'bip_swung_perc': 35.13513513513514,
  'taken_strike_perc': 24.0,
  'est_woba': 0.5106153846153847,
  'babip': 0.10526315789473684,
  'iso_value': 0.5263157894736842},
 'breaking': {'perc_faced': 24.551724137931036,
  'chase_perc': 23.423423423423422,
  'bip_swung_perc': 30.64516129032258,
  'taken_strike_perc': 46.26865671641791,
  'est_woba': 0.37794736842105264,
  'babip': 0.20588235294117646,
  'iso_value': 0.08823529411764706}}

In [13]:
len(batters_dict)

957

In [21]:
batters_df = pd.DataFrame.from_dict(batters_dict, orient='index')
batters_df = batters_df.reset_index().rename(columns={'index':'batter'}).head()


,batter,pitchout,fastball,offspeed,breaking
0,112526.0,{'perc_faced': 0},"{'perc_faced': 64.0625, 'chase_perc': 47.36842...","{'perc_faced': 1.5625, 'chase_perc': 100.0, 'b...","{'perc_faced': 34.375, 'chase_perc': 73.333333..."
1,134181.0,{'perc_faced': 0},"{'perc_faced': 61.026615969581755, 'chase_perc...","{'perc_faced': 8.17490494296578, 'chase_perc':...","{'perc_faced': 30.798479087452474, 'chase_perc..."
2,136860.0,{'perc_faced': 0},"{'perc_faced': 62.18879668049793, 'chase_perc'...","{'perc_faced': 11.20331950207469, 'chase_perc'...","{'perc_faced': 26.607883817427386, 'chase_perc..."
3,150029.0,{'perc_faced': 0.2304147465437788},"{'perc_faced': 65.3609831029186, 'chase_perc':...","{'perc_faced': 10.061443932411676, 'chase_perc...","{'perc_faced': 24.347158218125962, 'chase_perc..."
4,276520.0,{'perc_faced': 0},"{'perc_faced': 83.33333333333334, 'chase_perc'...","{'perc_faced': nan, 'chase_perc': nan, 'bip_sw...","{'perc_faced': 16.666666666666664, 'chase_perc..."


In [16]:
batters_df.isna().sum()

pitchout    0
fastball    0
offspeed    0
breaking    0
dtype: int64

## Next steps:

-expand each pitch cat dictionary key to a new column 

    -example: fast_ball_perc_faced, fastball_chase_perc ...

-merge batters_df, one to many, with the main df (df_17 in this case)
    - join on df['batter'] == batters_df['batter']